## Transform Customer data and load into Silver Layer
1. Remove records with NULL customer_id.
1. Remove exact duplicate records.
1. Remove duplicate records based on create_Timestamp.
1. Cast the column to the correct data type.
1. Write the transformed data to the Silver Layer.


#### 1. Remove records with NULL customer_id.


In [0]:
select * from gizmobox.bronze.v_customers
where customer_id is not null;

#### 2. Remove exact duplicate records.

In [0]:
select * from gizmobox.bronze.v_customers
where customer_id is not null
order by customer_id

#### Using Hash functions and CONCAT_WS ( concatinate with seperator)

In [0]:

  select * from (
select  SHA1(CONCAT_WS('||', *)) AS hash_val, customer_id, row_number() over(partition by SHA1(CONCAT_WS('||', *)) order by SHA1(CONCAT_WS('||', *))) as rowid
    FROM gizmobox.bronze.v_customers
    ) A
where rowid = 2;

create table gizmobox.bronze.customers_test select *  from gizmobox.bronze.v_customers;

select * from gizmobox.bronze.customers_test;

  select * from (
select  SHA1(CONCAT_WS('||', *)) AS hash_val, customer_id, row_number() over(partition by SHA1(CONCAT_WS('||', *)) order by SHA1(CONCAT_WS('||', *))) as rowid
    FROM gizmobox.bronze.customers_test
    ) A
where rowid = 2;

In [0]:
select concat_ws('||', *),sha1(concat_ws('||', *)) from gizmobox.bronze.customers_test


In [0]:
with duplicate as (
    select * from (
select  SHA1(CONCAT_WS('||', *)) AS hash_val, customer_id, row_number() over(partition by SHA1(CONCAT_WS('||', *)) order by SHA1(CONCAT_WS('||', *))) as rowid
    FROM gizmobox.bronze.customers_test
    ) A
where rowid = 2
)

delete from 

In [0]:
create or replace temp view v_customers_distinct as
select distinct * from gizmobox.bronze.v_customers
where customer_id is not null
order by customer_id;

In [0]:
select 
customer_id,
max(created_timestamp) 
from v_customers_distinct 
group by customer_id

#### 3. Remove duplicate records based on create_Timestamp.

In [0]:
with cte_max as (
select 
  customer_id,
  max(created_timestamp) as max_timestamp
from v_customers_distinct 
group by customer_id
)
select * from v_customers_distinct a
inner join cte_max b
on a.customer_id = b.customer_id
and a.created_timestamp = b.max_timestamp

#### 4. Cast the column to the correct data type.


In [0]:
with cte_max as (
select 
  customer_id,
  max(created_timestamp) as max_timestamp
from v_customers_distinct 
group by customer_id
)
select 
  cast(a.created_timestamp as timestamp) as created_timestamp,
  a.customer_id,
  a.customer_name,
  cast(a.date_of_birth as date) as date_of_birth,
  a.email,
  cast(a.member_since as date) as member_since,
  a.telephone 
from v_customers_distinct a  
inner join cte_max b
on a.customer_id = b.customer_id
and a.created_timestamp = b.max_timestamp

#### 5. Write the transformed data to the Delta table in Silver Layer.

In [0]:
CREATE TABLE gizmobox.silver.customers as
with cte_max as (
select 
  customer_id,
  max(created_timestamp) as max_timestamp
from v_customers_distinct 
group by customer_id
)
select 
  cast(a.created_timestamp as timestamp) as created_timestamp,
  a.customer_id,
  a.customer_name,
  cast(a.date_of_birth as date) as date_of_birth,
  a.email,
  cast(a.member_since as date) as member_since,
  a.telephone 
from v_customers_distinct a  
inner join cte_max b
on a.customer_id = b.customer_id
and a.created_timestamp = b.max_timestamp

In [0]:
select * from gizmobox.silver.customers